In [1]:
%load_ext sql
%sql sqlite:///PS1.db

Problem Set 1 [100 points]
=======

### Deliverables:

Submit your queries (and only those) using the `submission_template.txt` file that is posted on Canvas. Follow the instructions on the file! Upload the file at Canvas (under PS1).


### Instructions / Notes:

* Run the top cell above to load the database `PS1.db` (make sure the database file, `PS1.db`, is in the same directory as this IPython notebook is running in)
* Some of the problems involve _changing_ this database (e.g. deleting rows)- you can always re-download `PS1.db` or make a copy if you want to start fresh!
* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* _Have fun!_

Problem 1: Linear Algebra [25 points]
------------------------

Two random 3x3 ($N=3$) matrices have been provided in tables `A` and `B`, having the following schema:
> * `i INT`:   Row index
> * `j INT`:   Column index
> * `val INT`: Cell value

**Note: all of your answers below _must_ work for any _square_ matrix sizes, i.e. any value of $N$**.

Note how the matrices are represented- why do we choose this format?  Run the following queries to see the matrices in a nice format:

In [ ]:
%sql SELECT group_concat(val, ' , ') AS 'A' FROM A GROUP BY i;

In [ ]:
%sql SELECT group_concat(val, ' , ') AS 'B' FROM B GROUP BY i;

### Part (a): Matrix transpose [5 points]

_Transposing_ a matrix $A$ is the operation of switching its rows with its columns- written $A^T$.  For example, if we have:

$A=\begin{bmatrix}a & b & c\\ d & e & f\\ g & h & i\end{bmatrix}$

Then:

$A^T=\begin{bmatrix}a & d & g\\ b & e & h\\ c & f & i\end{bmatrix}$

Write a _single SQL query_ to get the matrix transpose $A^T$ (in the same format as $A$- output tuples should be of format `(i,j,val)` where `i` is row, `j` is column, and the output is ordered by row then column index)

Write your query here:

In [ ]:
%%sql
SELECT group_concat(val, ' , ') AS 'A-transpose'
FROM A
GROUP BY j
;
select A.j as i, A.i as j, A.val
from A
order by i
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (b): Dot product I [5 points]

The _dot product_ of two vectors

$a = \begin{bmatrix}a_1 & a_2 & \dots & a_n\end{bmatrix}$

and

$b = \begin{bmatrix}b_1 & b_2 & \dots & b_n\end{bmatrix}$

is

$a\cdot b = \sum_{i=1}^n a_ib_i = a_1b_1 + a_2b_2 + \dots + a_nb_n$

Write a _single SQL query_ to take the dot product of the **third column of $A$** and the **second column of $B$.**

Write your query here:

In [ ]:
%%sql
select sum(ATEMP.val * BTEMP.val) as 'Dot Product'
from (select * from A where j = 2) as ATEMP,
     (select * from B where j = 1) as BTEMP
where ATEMP.i = BTEMP.i
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (c): Matrix multiplication [10 points]

The product of a matrix $A$ (having dimensions $n\times m$) and a matrix $B$ (having dimensions $m\times p$) is the matrix $C$ (of dimension $n\times p$) having cell at row $i$ and column $j$ equal to:

$C_{ij} = \sum_{k=1}^m A_{ik}B_{kj}$

In other words, to multiply two matrices, get each cell of the resulting matrix $C$, $C_{ij}$, by taking the _dot product_ of the $i$th row of $A$ and the $j$th column of $B$.

Write a single SQL query to get the matrix product of $A$ and $B$ (in the same format as $A$ and $B$).

Write your query here:

In [ ]:
%%sql
select Atemp.i, Btemp.j, SUM(Atemp.val * Btemp.val) as val
from A Atemp
inner join B Btemp
on Atemp.j = Btemp.i
group by Atemp.i, Btemp.j
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (d): Matrix power [5 points]

The power $A^n$ of a matrix $A$ is defined as the matrix product of $n$ copies of $A$. 

Write a _single SQL query_ that computes the **third power** of matrix $A$ (in the same format as $A$), in other words, $A^3 = A \cdot A \cdot A$.

Write your query here:

In [ ]:
%%sql
select Atemp.i, Btemp.j, SUM(Atemp.val * Btemp.val) as val
from (select Atemp.i, Btemp.j, SUM(Atemp.val * Btemp.val) as val
        from A Atemp
        inner join A Btemp
        on Atemp.j = Btemp.i
        group by Atemp.i, Btemp.j) Atemp
inner join A Btemp
on Atemp.j = Btemp.i
group by Atemp.i, Btemp.j
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

Problem 2: The Sales Database [35 points]
----------------------------------------------

We've prepared and loaded a dataset related to sales data from a company. The dataset has the following schema:

> `Holidays (WeekDate, IsHoliday)`

> `Stores (Store, Type, Size)`

> `TemporalData(Store, WeekDate, Temperature, FuelPrice, CPI, UnemploymentRate)`

> `Sales (Store, Dept, WeekDate, WeeklySales)`

Before you start writing queries on the database, find the schema and the constraints (keys, foreign keys). 

### Part (a): Sales during Holidays [5 points]

Using a single SQL query, find the stores with the largest and smallest overall sales during holiday weeks. Further requirements:
* Use the `WITH` clause before the main body of the query to compute a subquery if necessary.
* Return a relation with schema `(Store, AllSales)`.

Write your query here:

In [ ]:
%%sql
with temp AS (
    select holSales.store, SUM(holSales.weeklysales) as 'Allsales'
        from (select sales.*
                from Sales sales,
                     Holidays hol
                where sales.WeekDate = hol.WeekDate and hol.IsHoliday = 'TRUE') holSales
        group by holSales.store)
select temp.store, max(temp.Allsales) as 'AllSales' from temp
union all
select temp.store, min(temp.Allsales) as 'AllSales' from temp
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (b): When Holidays do not help Sales [10 points]

Using a single SQL query, compute the **number** of non-holiday weeks that had larger sales than the overall average sales during holiday weeks. Further requirements:
* Use the `WITH` clause before the main body of the query to compute a subquery if necessary.
* Return a relation with schema `(NumNonHolidays)`.

Write your query here:

In [ ]:
%%sql
with holSales AS (
    select sales.*
    from Sales sales,
         Holidays hol
    where sales.WeekDate = hol.WeekDate and hol.IsHoliday = 'TRUE'),
nonHolSales AS (
    select sales.*
    from Sales sales,
         Holidays hol
    where sales.WeekDate = hol.WeekDate and hol.IsHoliday = 'FALSE'),
whst AS (
    select holSales.WeekDate, sum(holSales.weeklysales) as 'WeeklyTotal'
            from holSales
    group by holSales.WeekDate),
whavg AS (
    select avg(whst.WeeklyTotal) as 'average'
    from whst),
wst AS (
    select nonHolSales.store, nonHolSales.WeekDate, sum(nonHolSales.weeklysales) as 'WeeklyStoreTotal'
            from nonHolSales
    group by nonHolSales.store, nonHolSales.WeekDate),
wt AS (
    select wst.WeekDate, SUM(wst.WeeklyStoreTotal) as 'weeksum'
    from wst
    group by wst.WeekDate)
select count(*) as 'NumNonHolidays'
from whavg, wt
where wt.weeksum > whavg.average
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (c): Total Sales [5 points]

Using a _single SQL query_, compute the total sales during summer (months 6,7and 8) for each type of store. Further requirements:
* Return a relation with schema `(type, TotalSales)`.

*Hint:* SQLite3 does not support native operations on the DATE datatype. To create a workaround, you can use the `LIKE` predicate and the string concatenation operator (||). You can also use the substring operator that SQLite3 supports (`substr`).

Write your query here:

In [ ]:
%%sql
with store_sales AS (
    select sales.store, sales.WeekDate, sum(sales.weeklysales) as 'sales'
    from Sales sales
    where sales.WeekDate like '%-06-%' or
          sales.WeekDate like '%-07-%' or
          sales.WeekDate like '%-08-%'
    group by sales.store),
storetype_sales AS (
    select store_sales.store, stores.type, store_sales.sales
    from store_sales, Stores stores
    where store_sales.store = stores.store)
select storetype_sales.type, sum(storetype_sales.sales)
from storetype_sales
group by storetype_sales.type
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (d): Computing Correlations [15 points]

The goal of this exercise is to find out whether each one of the 4 numeric attributes in `TemporalData` is
positively or negatively correlated with sales.

For our purposes, the intuitive notion of correlation is defined using a
standard statistical quantity known as the *Pearson correlation coefficient*.
Given two numeric random variables $X$ and $Y$, it is defined as follows:

$$\rho_{X,Y} = \frac{E[XY] - E[X]E[Y]}{\sqrt{E[X^2] - E[X]^2} \sqrt{E[Y^2] - E[Y]^2}}$$

On a given sample of data with $n$ examples each for $X$ and $Y$ (label them
$x_i$ and $y_i$ respectively for $i = 1 \dots n$), it is estimated as follows:

\begin{align*}
\rho_{X,Y} = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2} \sqrt{\sum_{i=1}^n (y_i - \bar{y})^2}}
\end{align*}
In the above, $\bar{x}$ and $\bar{y}$ are the sample means, i.e.,
$\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i$, and $\bar{x} = \frac{1}{n}\sum_{i=1}^n y_i$.

Further requirements:
* Each example in the sample consists of a pair (Store, WeekDate). This means that Sales data must be summed across all departments before the correlation is computed. 
* Return a relation with schema `(AttributeName VARCHAR(20), CorrelationSign Integer)`. 
* The values of AttributeName can be hardcoded string literals, but the values
of `CorrelationSign` must be computed automatically using SQL queries over
the given database instance.
* You can use multiple SQL statements to compute the result. It might be of help to create and update your own tables.

Write your query here:

In [ ]:
%%sql
drop table if exists results;
create table results (
    AttributeName VARCHAR(20),
    CorrelationSign Integer
);
drop table if exists temp_comp;
create table temp_comp AS
    with dataset AS (
        with sum_sales_across_dept AS (
            select sales.Store, sales.WeekDate, sum(sales.WeeklySales) as 'ws'
            from Sales sales
            group by sales.Store, sales.WeekDate)
        select meta.Store, meta.WeekDate, ssad.ws, meta.Temperature, meta.FuelPrice, meta.CPI, meta.UnemploymentRate
        from sum_sales_across_dept ssad, TemporalData meta
        where ssad.Store = meta.Store and ssad.WeekDate = meta.WeekDate),
    store_avgs AS (
        select
               avg(ds.ws) as 'avg_ws',
               avg(ds.Temperature) as 'avg_temp',
               avg(ds.FuelPrice) as 'avg_fp',
               avg(ds.CPI) as 'avg_cpi',
               avg(ds.UnemploymentRate) as 'avg_ur'
        from dataset ds),
    computation AS (
    select sum((ds.Temperature-sa.avg_temp)*(ds.ws-sa.avg_ws)) as 'nt',
           sum((ds.FuelPrice-sa.avg_fp)*(ds.ws-sa.avg_ws)) as 'nfp',
           sum((ds.CPI-sa.avg_cpi)*(ds.ws-sa.avg_ws)) as 'ncpi',
           sum((ds.UnemploymentRate-sa.avg_ur)*(ds.ws-sa.avg_ws)) as 'nur'
    from store_avgs sa, dataset ds)
    select * from computation
;
insert into results
    select
    'Temperature' as 'AttributeName',
    case
        when t.nt > 0 then 1
        when t.nt < 0 then -1
        else 0
        END CorrelationSign
from temp_comp t;
insert into results
    select
    'FuelPrice' as 'AttributeName',
    case
        when t.nfp > 0 then 1
        when t.nfp < 0 then -1
        else 0
        END CorrelationSign
from temp_comp t;
insert into results
    select
    'CPI' as 'AttributeName',
    case
        when t.ncpi > 0 then 1
        when t.ncpi < 0 then -1
        else 0
        END CorrelationSign
from temp_comp t;
insert into results
    select
    'UnemploymentRate' as 'AttributeName',
    case
        when t.nur > 0 then 1
        when t.nur < 0 then -1
        else 0
        END CorrelationSign
from temp_comp t;
select * from results;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

Problem 3: The Traveling SQL Server Salesman Problem [40 points]
--------------------------------------------------

SQL Server salespeople are lucky as far as traveling salespeople go- they only have to sell one or two big enterprise contracts, at one or two offices in Wisconsin, in order to make their monthly quota!

Answer the following questions using the table of streets connecting company office buildings.

**Note that for convenience all streets are included _twice_, as $A \rightarrow B$ and $B \rightarrow A$.  This should make some parts of the problem easier, but remember to take it into account!**

In [ ]:
%sql SELECT * FROM streets LIMIT 4;

### Part (a): One-hop, two-hop, three-hop... [10 points]

Our salesperson has stopped at UW-Madison, to steal some cool new RDBMS technology from CS564-ers, and now wants to go sell it to a company _within 10 miles of UW-Madison and _passing through no more than 3 distinct streets_.  Write a single query, not using `WITH` (see later on), to find all such companies.

Your query should return the schema `(company, distance)` where distance is cumulative from UW-Madison.

Write your query here:

In [ ]:
%%sql
select s1.B as 'company', (s1.d) as 'distance'
from streets s1
where s1.A = 'UW-Madison' and s1.d <= 10
union all
select s2.B as 'company', (s1.d + s2.d) as 'distance'
from streets s1, streets s2
where s1.A = 'UW-Madison' and s1.B = s2.A and (s1.d + s2.d) <= 10
union all
select s3.B as 'company', (s1.d + s2.d + s3.d) as 'distance'
from streets s1, streets s2, streets s3
where s1.A = 'UW-Madison' and s1.B = s2.A and s2.B = s3.A and (s1.d + s2.d + s3.d) <= 10
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (b): A stop at the Farm [10 points]

Now, our salesperson is out in the field, and wants to see all routes- and their distances- which will take him/her from a company $A$ to a company $B$, with the following constraints:
* The route must _pass through UW-Madison (in order to pick up new RDBMS tech to sell!)
* $A$ and $B$ must _each individually_ be _within 3 hops of UW-Madison
* $A$ and $B$ must be different companies
* _The total distance must be $<= 15$_
* Do not use `WITH`
* If you return a path $A \rightarrow B$, _also include $B \rightarrow A$ in your answer_

In order to make your answer a bit cleaner, you may split into two queries, one of which creates a `VIEW`.  A view is a virtual table based on the output set of a SQL query.  A view can be used just like a normal table- the only difference under the hood is that the DBMS re-evaluates the query used to generate it each time a view is queried by a user (thus the data is always up-to date!)

Here's a simple example of a view:

In [ ]:
%%sql 
DROP VIEW IF EXISTS short_streets;
CREATE VIEW short_streets AS 
SELECT A, B, d FROM streets WHERE d < 3;
SELECT * FROM short_streets LIMIT 3;

Write your query or queries here:

In [ ]:
%%sql
drop view if exists madison_close;
create view madison_close AS
    select distinct s1.A as 'A', s1.B as 'B', s1.d as 'd'
    from streets s1
    where s1.A != 'UW-Madison' and s1.B = 'UW-Madison' and s1.d <= 15
    union all
    select distinct s1.A as 'A', s2.B as 'B', (s1.d + s2.d) as 'd'
    from streets s1, streets s2
    where s1.B = s2.A and s1.A != 'UW-Madison' and s1.B != 'UW-Madison' and s2.A != 'UW-Madison' and s2.B = 'UW-Madison' and (s1.d + s2.d) <= 15
    union all
    select distinct s1.A as 'A', s3.B as 'B', (s1.d + s2.d + s3.d) as 'd'
    from streets s1, streets s2, streets s3
    where s1.B = s2.A and s2.B = s3.A and s2.B != s1.A and s1.A != 'UW-Madison' and s1.B != 'UW-Madison' and s2.A != 'UW-Madison' and s2.B != 'UW-Madison' and s3.A != 'UW-Madison' and s3.B = 'UW-Madison' and (s1.d + s2.d + s3.d) <= 15
;
select distinct c1.A as 'company_1', c2.A as 'company_2', (c1.d + c2.d) as 'distance'
from madison_close c1, madison_close c2
where c1.A != c2.A and c1.A != 'UW-Madison' and c1.B = 'UW-Madison' and c2.A != 'UW-Madison' and c2.B = 'UW-Madison' and (c1.d + c2.d) <= 15
order by distance
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (c): Ensuring acyclicity [10 points]

You may have noticed at this point that the network, or **_graph_**, of streets in our `streets` table seems like it might be a **tree**.

Recall that a **_tree_** is an undirected graph where each node has exactly one parent (or, is the root, and has none), but may have multiple children.  A slightly more formal definition of a tree is as follows: 

> An undirected graph $T$ is a **_tree_** if it is **connected**- meaning that there is a path between every pair of nodes- and has no **cycles** (informally, closed loops)

Suppose that we guarantee the following about the graph of companies & streets determined by the `streets` table:
* It is _connected_
* It has no cycles of length $> 3$

Write a _single SQL query_ which, if our graph is not a tree (i.e. if this isn't a [shaggy dog problem](https://en.wikipedia.org/wiki/Shaggy_dog_story)), **turns it into a tree** by deleting exactly _one_ street (*= two entries in our table!*).  Write your query here:

In [ ]:
%%sql
delete from streets
where id in (
    select s1.id
    from streets s1, streets s2, streets s3
    where s1.B = s2.A and s2.B = s3.A and s3.B = s1.A
    limit 1)
;

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (d): The longest path [10 points]

In this part, we want to find the distance of the _longest path between any two companies_.

Note that you should probably have done Part (c) first so that the graph of streets is a _tree_- this will make it much easier to work with!

If you've done the other parts above, you might be skeptical that SQL can find paths of arbitrary lengths (which is what we need to do for this problem); how can we do this?

There are some non-standard SQL functions- i.e. not universally supported by all SQL DBMSs- which are often incredibly useful.  One of these is the `WITH RECURSIVE` clause, supported by SQLite.

A `WITH` clause lets you define what is essentially a view within a clause, mainly to clean up your SQL code.  A trivial example, to illustrate `WITH`:

In [ ]:
%%sql
WITH companies(name) AS (
    SELECT DISTINCT A FROM streets)
SELECT * 
FROM companies 
WHERE name LIKE '%Gizmo%';

There is also a recursive variant, `WITH RECURSIVE`.  `WITH RECURSIVE` allows you to define a view just as above, except the table can be defined recursively.  A `WITH RECURSIVE` clause must be of the following form:

```
WITH RECURSIVE 
    R(...) AS (
        SELECT ... 
        UNION [ALL] 
        SELECT ... FROM R, ...
    )
...
```

`R` is the _recursive table_.  The `AS` clause contains two `SELECT` statements, conjoined by a `UNION` or `UNION ALL`; the first `SELECT` statement provides the initial / base case values, and the second or _recursive_ `SELECT` statement must include the recursive table in its `FROM` clause.

Basically, the recursive `SELECT` statement continues executing on the tuple _most recently inserted into `R`_, inserting output rows back into `R`, and proceeding recursively in this way, until it no longer outputs any rows, and then stops.  See the [SQLite documentation](https://www.sqlite.org/lang_with.html) for more detail.

The following example computes $5! = 5*4*3*2*1$ using `WITH RECURSIVE`:

In [ ]:
%%sql
WITH RECURSIVE
    factorials(n,x) AS (
        SELECT 1, 1
        UNION
        SELECT n+1, (n+1)*x FROM factorials WHERE n < 5)
SELECT x FROM factorials WHERE n = 5;

In this example:

1. First, `(1,1)` is inserted into the table `factorials` (the base case).
2. Next, this tuple is returned by the recursive select, as `(n, x)`, and we insert the result back into `factorials`: `(1+1, (1+1)*1) = (2,2)`
3. Next, we do the same with the last tuple inserted into `factorials`- `(2,2)`- and insert `(2+1, (2+1)*2) = (3,6)`
4. And again: get `(3,6)` from `factorials` and insert `(3+1, (3+1)*6) = (4,24)` back in
5. And again: `(4,24)` -> `(4+1, (4+1)*24) = (5,120)`
6. Now the last tuple inserted into `factorials` is `(5, 120)`, which fails the `WHERE n < 5` clause, and thus our recursive select returns an empty set, and our `WITH RECURSIVE` statement concludes
7. Finally, now that our `WITH RECURSIVE` clause has concluded, we move on to the `SELECT x FROM factorials WHERE n=5` clause, which gets us our answer!

#### Now, your turn!

Write a single SQL query that uses `WITH RECURSIVE` to find the furthest (by distance) pair of companies that still have a path of streets connecting them.  Your query should return `(A, B, distance)`.

Write your query here:

In [7]:
%%sql
with recursive
    longestboi(A, B, distance, start) as (
        select s.A, s.B, s.d, s.A
        from streets s
        union
        select s2.A, s2.B, s2.d + lb.distance, lb.start
        from streets s2, longestboi lb
        where s2.A = lb.B and not s2.B = lb.A and not lb.start = s2.B)
select lb.start as 'A', lb.B, lb.distance
from longestboi lb
order by lb.distance desc
limit 1;

 * sqlite:///PS1.db
Done.


A,B,distance
GadgetWorks,ThingWorks,63


In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Note on alternate output

**NOTE:** The **_distance_** of the longest path could be **49 _OR_ 63** depending on which street you deleted in Part (c)!